# Orbit Determination Pipeline

This tutorial demonstrates how to perform orbit determination.

## Setup

In [21]:
import numpy as np
import pandas as pd

import plotly.graph_objs as go

from ostk.mathematics.objects import RealInterval

from ostk.physics.units import Length
from ostk.physics.units import Derived
from ostk.physics.units import Angle
from ostk.physics.units import Time
from ostk.physics.time import Scale
from ostk.physics.time import Instant
from ostk.physics.time import Duration
from ostk.physics.time import Interval
from ostk.physics.time import DateTime
from ostk.physics.coordinate import Position
from ostk.physics.coordinate import Velocity
from ostk.physics.coordinate.spherical import LLA
from ostk.physics.coordinate import Frame
from ostk.physics import Environment
from ostk.physics.environment.objects.celestial_bodies import Earth

from ostk.astrodynamics import Trajectory
from ostk.astrodynamics.trajectory import State
from ostk.astrodynamics.trajectory import Orbit
from ostk.astrodynamics.trajectory import Model
from ostk.astrodynamics.trajectory.orbit import Pass
from ostk.astrodynamics.trajectory.orbit.models import Propagated

## Add extra imports

---

## Processing GPS Data and Related Telemetry

In [ ]:
gps_measurements_timestamped =DataVector('file location')
gps_receiver_noise = Matrix('file with gps receiver sigmas for position and velocity')

# potentially running through smoother/discarding data holes and providing statistics on quality/continuity of data

---

## Batch Least Squares Estimation (for orbital state)

In [ ]:
current_instant = Instant.Undefined()

frame = Frame.GCRF()
position = Position.Undefined()
velocity = Velocity.Undefined()

undefined_state = State(current_instant, position, velocity)

In [ ]:
orbital_state_estimation = Estimation(Environment.default(), Duration('3 weeks'), ConfigType('Position and Velocity estimation'))

system = System(Mass, Environment.default(), undefined_state, System.GraviationalPerturbatioType.FourtyByFourty, System.AtmosphericPerturbatioType.NRLMSIS00, System.ThirdBodyPerturbatioType.LuniSolar)
solver = Solver(Solver.RungeKuttaCashKarp54, Solver.NoLog, 1.0e-15, 1.0e-15)

ecef_to_eci = DynamicFrameProvider()

measurement_model = MeasurementModel(MeasurementModel.OnBoardGPSMeas, ecef_to_eci)

state_apriori = StateEstimate(DataVector([0]), CovarianceMatrix) # grab the first GPS measurement from the data vector to use as the apriori, after it has been transformed to ECI

batch_least_squares = BatchLeastSquares(state_apriori, system, solver, gps_measurements_timestamped, measurement_model, gps_receiver_noise)

initial_state_estimate = batch_least_squares.estimate_state_with_iterations(Integer)

---

## Batch Least Squares Estimation (for other system related parameters like drag)

In [ ]:
# todo once orbital state estimation is well understood

---

## Orbit Propagation/Prediction to final state

Computation

In [2]:
earth_env = Environment.default().access_celestial_object_with_name("Earth")

Feed state estimate into system 

In [3]:

system = System(Mass, Environment.default(), initial_state_estimate, System.GraviationalPerturbatioType.FourtyByFourty, System.AtmosphericPerturbatioType.NRLMSIS00, System.ThirdBodyPerturbatioType.LuniSolar)
solver = Solver(Solver.RungeKuttaCashKarp54, Solver.NoLog, 1.0e-15, 1.0e-15)

Setup a Propagated orbital model 

In [4]:

propagated_model_estimate = Propagated(earth_env, system, solver) 

Setup the orbit:

In [5]:
orbit_default = Orbit(propagated_model_estimate, earth_env)
# orbit_custom = Orbit(propagated_model_custom, earth_env)

Now that the orbit is set, we can compute the satellite position a certain amount of time in the future:

In [6]:
current_instant = Instant.now()

predicted_state_estimate = orbit_default.get_state_at(current_instant + orbital_state_estimation.getTimeHorizon()) 

In [ ]:
# extra logic to handle position covariance and translate it to acess uncertainty and ground coverage uncertainty to feed back towards the customers
# potentially implement some sort of uncertainty metric in cockpit as customers task the spacecraft further into the future 

---